In [105]:
#!/usr/bin/env python
# -*- coding: latin-1 -*-
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import csv
import logging
from SPARQLWrapper import SPARQLWrapper,JSON, SPARQLWrapper2, POST,XML,GET
from collections import defaultdict
from tqdm.notebook import tqdm
import query_helper_functions as q_helper
import sparql_query as sparql_query
import imp
imp.reload(q_helper)
imp.reload(sparql_query)


<module 'sparql_query' from 'C:\\Users\\rania\\OneDrive\\AI Master\\Project\\Python_Code\\Final_Code\\modules\\sparql_query.py'>

In [106]:

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
tqdm.pandas(desc="progress bar")

d:\programs\python39\lib\site-packages\tqdm\std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [107]:
ml_file_movies_df = pd.read_csv('dataset/ml_movies.csv')

In [108]:
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 20)
#pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-108-3a6f8d9880c9>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [109]:
ml_file_movies_df.drop(columns='movieId',inplace=True)

In [110]:
#clean the titles in ml data
# extract the year from the title and save it in a new column
ml_file_movies_df['year'] =  ml_file_movies_df.title.str.extract(r'\(([0-9]+)\)')
# remove any parantatheis from the title
ml_file_movies_df['title'] = ml_file_movies_df['title'].str.replace(r"\(.*\)","")
#remove white spaces from the title and convert it to lower case
ml_file_movies_df.title = ml_file_movies_df.title.str.strip().str.lower()
#remove double qoutation from title
ml_file_movies_df['title'] = ml_file_movies_df['title'].str.replace("\"","")
#move "the" from the end of the title to the begining
ml_file_movies_df['title'] = ml_file_movies_df['title'].apply(lambda row: 'the ' + row.replace(", the", "") if ", the" in row else row)

In [111]:
#copy the movies into new dataframe
ml_movies_df = ml_file_movies_df.copy()
ml_movies_df.describe

<bound method NDFrame.describe of                              title  \
0      toy story                     
1      jumanji                       
2      grumpier old men              
3      waiting to exhale             
4      father of the bride part ii   
...                            ...   
62418  we                            
62419  window of the soul            
62420  bad poems                     
62421  a girl thing                  
62422  women of devil's island       

                                            genres  year  
0      Adventure|Animation|Children|Comedy|Fantasy  1995  
1      Adventure|Children|Fantasy                   1995  
2      Comedy|Romance                               1995  
3      Comedy|Drama|Romance                         1995  
4      Comedy                                       1995  
...       ...                                        ...  
62418  Drama                                        2018  
62419  Documentary                   

In [112]:
ml_movies_df[30000:30050]

,title,genres,year
30000,blue summer,Comedy|Drama,1973
30001,hamari adhuri kahaani,(no genres listed),2015
30002,such good people,Comedy|Mystery,2014
30003,the possibilities are endless,Documentary,2014
30004,mostly ghostly: have you met my ghoulfriend?,Children|Fantasy,2014
30005,jailbait babysitter,Comedy,1977
30006,pocket ninjas,Action|Children,1997
30007,occupy love,Documentary|Drama,2013
30008,the sky above us,Drama|War,2015
30009,prime suspect 5: errors of judgement,Crime|Drama,1996


In [113]:
#extract titles list to be used for querying LOD
movies_list = ml_movies_df['title'].values.tolist()

In [114]:
print("Number of movies from MovieLens dataset is {} ".format(len(movies_list)))

Number of movies from MovieLens dataset is 62423 


### Query LOD for movies info (dbpedia movie link, dbpedia title, yago movie link, yago writer name, dbpedia or yago director_name, yago country, yago genre) 

In [115]:
#uncomment either the call to query lod to refresh the data or the load from file
endpoint = "http://lod.openlinksw.com/sparql/query"
return_format = XML
request_method= POST
chunck_size = 2000
movies_info_result_df = q_helper.retrieve_data_for_movies_list(sparql_query.lod_movies_info_query, movies_list, chunck_size,'string', endpoint, return_format, request_method)
movies_info_df = movies_info_result_df[0].copy()
failed_movies_info_list = movies_info_result_df[1].copy()
#save raw results of movies info query to file
movies_info_df.to_csv("generated_dataset/lod_movies_info_query_raw_results_1.csv",index=False)

Searching by movie title
Processing movies between 0 2000

HTTP Error 405: Not Allowed
Chunck Failed : """toy story""","""jumanji""","""grumpier old men""","""waiting to exhale""","""father of the bride part ii""","""heat""","""sabrina""","""tom and huck""","""sudden death""","""goldeneye""","""the american president""","""dracula: dead and loving it""","""balto""","""nixon""","""cutthroat island""","""casino""","""sense and sensibility""","""four rooms""","""ace ventura: when nature calls""","""money train""","""get shorty""","""copycat""","""assassins""","""powder""","""leaving las vegas""","""othello""","""now and then""","""persuasion""","""the city of lost children""","""shanghai triad""","""dangerous minds""","""twelve monkeys""","""wings of courage""","""babe""","""carrington""","""dead man walking""","""across the sea of time""","""it takes two""","""clueless""","""the cry beloved country""","""richard iii""","""dead presidents""","""restoration""","""mortal kombat""","""to die

In [60]:
#load previously retrieved data from file
movies_info_df= pd.read_csv("generated_dataset/lod_movies_info_query_raw_results.csv")

In [46]:
movies_info_df[1:5]

,pedia_movie,yago_movie,title,writer_name,director_name,country_id,genre_id
1,http://dbpedia.org/resource/In_the_Mouth_of_Ma...,http://yago-knowledge.org/resource/In_the_Mout...,In the Mouth of Madness,H. P. Lovecraft,John Carpenter,http://yago-knowledge.org/resource/United_States,http://yago-knowledge.org/resource/Horror_film
2,http://dbpedia.org/resource/In_the_Mouth_of_Ma...,http://yago-knowledge.org/resource/In_the_Mout...,In the Mouth of Madness,H. P. Lovecraft,John Carpenter,http://yago-knowledge.org/resource/United_States,http://yago-knowledge.org/resource/Fantasy_film
3,http://dbpedia.org/resource/In_the_Mouth_of_Ma...,http://yago-knowledge.org/resource/In_the_Mout...,In the Mouth of Madness,H. P. Lovecraft,John Carpenter,http://yago-knowledge.org/resource/United_States,http://yago-knowledge.org/resource/Thriller_film
4,http://dbpedia.org/resource/In_the_Mouth_of_Ma...,http://yago-knowledge.org/resource/In_the_Mout...,In the Mouth of Madness,H. P. Lovecraft,John Carpenter,http://yago-knowledge.org/resource/United_States,http://yago-knowledge.org/resource/Monster_movie


In [64]:
movies_info_df.pedia_movie.unique().size

13854

In [65]:


#extract the genre and country from yago link
movies_info_df['genre'] = movies_info_df.genre_id.str.split('http://yago-knowledge.org/resource/').str[1]
movies_info_df['country'] = movies_info_df.country_id.str.split('http://yago-knowledge.org/resource/').str[1]
#create movie id by extracting the dbpedia movie id from the link
movies_info_df['movie_id']= movies_info_df.pedia_movie.str.split('http://dbpedia.org/resource/').str[1]
movies_info_df = movies_info_df.fillna('NOT_FOUND')

In [49]:
movies_info_df.isna().any().sum()

0

In [66]:
#Group movies info results to extract the list of directors, writers, and titles
movies_info_grouped = movies_info_df.groupby(['movie_id','title'], as_index=False)
#aggregate the directors and directors in one column for each separated by |
lod_info_df = movies_info_grouped.agg({
            'director_name':lambda x: '|'.join(set(x)),
            'writer_name':lambda x: '|'.join(set(x)),
            'genre':lambda x: '|'.join(set(x)),
            'country':lambda x: '|'.join(set(x))
            })
#save grouped data to file
lod_info_df.to_csv("generated_dataset/lod_info_grouped.csv", index=False)

In [70]:
lod_info_df[lod_info_df.title == 'In the Mouth of Madness']

,movie_id,title,director_name,writer_name,genre,country
5487,In_the_Mouth_of_Madness,In the Mouth of Madness,John Carpenter,Michael De Luca|H. P. Lovecraft,Thriller_film|Monster_movie|Mystery_film|Fanta...,United_States


In [69]:
print("Number of ML movies retrieved from LOD with their info is {}".format(len(lod_info_df)))

Number of ML movies retrieved from LOD with their info is 13861


In [68]:
#load previously retrieved data from file
lod_info_df = pd.read_csv("generated_dataset/lod_info_grouped.csv")


In [71]:
lod_info_df[lod_info_df.director_name.str.contains('\(')]

,movie_id,title,director_name,writer_name,genre,country
16,100_Girls,100 Girls,Michael Davis (director),Michael Davis (director),Teen_film|Romantic_comedy,United_States
38,12_to_the_Moon,12 to the Moon,David Bradley (director),DeWitt Bodeen,Science_fiction_film,United_States
48,14_Blades,14 Blades,Daniel Lee (film director),Daniel Lee (film director),Martial_arts_film|Action_film,China
49,14_Going_on_30,14 Going on 30,Paul Schneider (director),Richard Jefferies|NOT_FOUND|James Orr (filmmaker),Science_fiction_film|Romantic_comedy|Science_f...,United_States
59,2001_Maniacs,2001 Maniacs,Tim Sullivan (director),Tim Sullivan (director)|Chris Kobin,Comedy_horror|Splatter_film,United_States
...,...,...,...,...,...,...
13752,You_Can't_Win_'Em_All,You Can't Win 'Em All,Peter Collinson (film director),Leo Gordon,War_film|Buddy_film,United_Kingdom
13754,You_Got_Served,You Got Served,Chris Stokes (record producer),Chris Stokes (record producer),Musical_film|Teen_film,United_States
13757,You_Lucky_Dog,You Lucky Dog,Paul Schneider (director),NOT_FOUND,Teen_film,United_States
13798,Z_Storm,Z Storm,David Lam (film director),NOT_FOUND,Crime_film,NOT_FOUND


### Query LOD for the actors of the ML movies

In [ ]:
#uncomment either the query to refresh the data from the source or load the previously retrieved data from file
endpoint = "http://lod.openlinksw.com/sparql/"
return_format = XML
request_method= POST
chunk_size = 2000
#movies_actors_results_df = q_helper.retrieve_data_for_movies_list(sparql_query.lod_actors_query,movies_list,chunk_size,'string',endpoint, return_format,request_method)
movies_actors_df= movies_actors_results_df[0].copy()
failed_movies_actors_list = movies_actors_results_df[1].copy()
#save movies actors results to file
movies_actors_df.to_csv("generated_dataset/lod_movies_actors_query_raw_results_yago.csv",index=False)


In [73]:
#Load from file
movies_actors_df = pd.read_csv("generated_dataset/lod_movies_actors_query_raw_results_yago.csv")

In [74]:
#create movie id by extracting the dbpedia movie id from the link
movies_actors_df['movie_id']= movies_actors_df.pedia_movie.str.split('http://dbpedia.org/resource/').str[1]

In [77]:
movies_actors_df

,pedia_movie,yago_movie,title,actor_name,movie_id
0,http://dbpedia.org/resource/Cliffhanger_(film),http://yago-knowledge.org/resource/Cliffhanger...,Cliffhanger,Paul Winfield,Cliffhanger_(film)
1,http://dbpedia.org/resource/Cliffhanger_(film),http://yago-knowledge.org/resource/Cliffhanger...,Cliffhanger,Michael Rooker,Cliffhanger_(film)
2,http://dbpedia.org/resource/Cliffhanger_(film),http://yago-knowledge.org/resource/Cliffhanger...,Cliffhanger,Janine Turner,Cliffhanger_(film)
3,http://dbpedia.org/resource/Cliffhanger_(film),http://yago-knowledge.org/resource/Cliffhanger...,Cliffhanger,Leon Robinson,Cliffhanger_(film)
4,http://dbpedia.org/resource/Cliffhanger_(film),http://yago-knowledge.org/resource/Cliffhanger...,Cliffhanger,Sylvester Stallone,Cliffhanger_(film)
...,...,...,...,...,...
82472,http://dbpedia.org/resource/Hungry_for_Monsters,http://yago-knowledge.org/resource/Hungry_for_...,Hungry for Monsters,Pascal Cervo,Hungry_for_Monsters
82473,http://dbpedia.org/resource/Staline:_le_tyran_...,http://yago-knowledge.org/resource/Staline:_le...,Staline: le tyran rouge,Pascal Cervo,Staline:_le_tyran_rouge
82474,http://dbpedia.org/resource/The_Painting,http://yago-knowledge.org/resource/The_Painting,The Painting,Pascal Cervo,The_Painting
82475,http://dbpedia.org/resource/The_Family_of_Chimps,http://yago-knowledge.org/resource/The_Family_...,The Family of Chimps,Pascal Cervo,The_Family_of_Chimps


In [80]:
movies_actors_df[movies_actors_df.title == 'Nine Months']

,pedia_movie,yago_movie,title,actor_name,movie_id
308,http://dbpedia.org/resource/Nine_Months,http://yago-knowledge.org/resource/Nine_Months,Nine Months,Tom Arnold (actor),Nine_Months
309,http://dbpedia.org/resource/Nine_Months,http://yago-knowledge.org/resource/Nine_Months,Nine Months,Hugh Grant,Nine_Months
310,http://dbpedia.org/resource/Nine_Months,http://yago-knowledge.org/resource/Nine_Months,Nine Months,Joan Cusack,Nine_Months
311,http://dbpedia.org/resource/Nine_Months,http://yago-knowledge.org/resource/Nine_Months,Nine Months,Julianne Moore,Nine_Months
312,http://dbpedia.org/resource/Nine_Months,http://yago-knowledge.org/resource/Nine_Months,Nine Months,Jeff Goldblum,Nine_Months
313,http://dbpedia.org/resource/Nine_Months,http://yago-knowledge.org/resource/Nine_Months,Nine Months,Robin Williams,Nine_Months


In [ ]:
"NOT_FOUND" in movies_actors_df.actor_name.values

In [81]:
#group the actors per movie and save data to file
movies_actors_grouped = movies_actors_df.groupby(['movie_id'], as_index=False)

#aggregate the actors in one column for each movie separated by |
lod_movies_actors_df = movies_actors_grouped.agg({'actor_name':lambda x: '|'.join(set(x))
                                         })

#pedia_actors_df.to_csv("generated_dataset/dbpedia_actors_grouped.csv",index=False)


In [ ]:
lod_movies_actors_df

In [85]:
lod_movies_actors_df[lod_movies_actors_df.movie_id == 'Nine_Months']

,movie_id,actor_name
9655,Nine_Months,Tom Arnold (actor)|Julianne Moore|Joan Cusack|Jeff Goldblum|Robin Williams|Hugh Grant


In [ ]:
"NOT_FOUND" in lod_movies_actors_df.actor_name.values

In [86]:
print("Number of ML movies retrieved from LOD with their actors is {}".format(len(lod_movies_actors_df)))

Number of ML movies retrieved from LOD with their actors is 19320


In [87]:
#Group movies info results to extract the list of directors, writers, and titles
movies_info_grouped = movies_info_df.groupby(['movie_id','title'], as_index=False)
#aggregate the directors and directors in one column for each separated by |
lod_info_df = movies_info_grouped.agg({
            'director_name':lambda x: '|'.join(set(x)),
            'writer_name':lambda x: '|'.join(set(x)),
            'genre':lambda x: '|'.join(set(x)),
            'country':lambda x: '|'.join(set(x))
            })

#group the actors per movie and save data to file
movies_actors_grouped = movies_actors_df.groupby(['movie_id'], as_index=False)
#aggregate the actors in one column for each movie separated by |
lod_movies_actors_df = movies_actors_grouped.agg({'actor_name':lambda x: '|'.join(set(x))
                                         })

#merge all dataframes to create a final dataset with the attributes, movie,title,yago_link,wiki_link,actor_name,writer_name,director_name,genre_id,country_id
final_movies_df = lod_info_df.merge(lod_movies_actors_df, how="inner", on=['movie_id'])


In [88]:
final_movies_df

,movie_id,title,director_name,writer_name,genre,country,actor_name
0,$9.99,$9.99,Tatia Rosenthal,Tatia Rosenthal|Etgar Keret,NOT_FOUND,Australia|Israel,Anthony LaPaglia|Geoffrey Rush
1,...All_the_Marbles,...All the Marbles,Robert Aldrich,NOT_FOUND,Comedy-drama|Action_film,United_States,Tracy Reed (American actress)|Burt Young|Laurene Landon|Peter Falk|Vicki Frederick
2,...First_Do_No_Harm,...First Do No Harm,Jim Abrahams,NOT_FOUND,NOT_FOUND,United_States,Lynne Griffin|Fred Ward|Tom Butler|Allison Janney|Leo Burmester|Ingrid Veninger|Jennifer Podemski|Meryl Streep|Margo Martindale|Seth Adkins
3,...ing,...ing,NOT_FOUND,NOT_FOUND,Romance_film,South_Korea,Kim Rae-won|Im Soo-jung (actress)|Lee Mi-sook
4,"1,_2,_3,_Sun","1, 2, 3, Sun",Bertrand Blier,Bertrand Blier,NOT_FOUND,France,Jean-Pierre Marielle|Marcello Mastroianni|Anouk Grinberg
...,...,...,...,...,...,...,...
12866,Zozo,Zozo,Josef Fares,Josef Fares,NOT_FOUND,Sweden|United_Kingdom|Lebanon,Carmen Lebbos
12867,Zubeidaa,Zubeidaa,Shyam Benegal,Khalid Mohamed,Romance_film,India,Karisma Kapoor|Farida Jalal|Rekha|Amrish Puri
12868,Zulu_Dawn,Zulu Dawn,Douglas Hickox,Cy Endfield|NOT_FOUND,War_film,United_Kingdom|South_Africa|United_States|Netherlands,John Mills|Ronald Pickup|Burt Lancaster|Denholm Elliott|Peter O'Toole|Ronald Lacey|Denholm Elliot|Simon Ward|Michael Jayston|Bob Hoskins
12869,Zum_Teufel_mit_der_Penne,Zum Teufel mit der Penne,Werner Jacobs,NOT_FOUND,NOT_FOUND,Germany,Balduin Baas|Joachim Teege|Willy Millowitsch|Hans Kraus|Rudolf Schündler|Hannelore Elsner|Peter Alexander|Heintje Simons|Theo Lingen|Sabine Bethmann|Johanna König


In [ ]:
#write final dataset to file
#final_movies_df.to_csv("generated_dataset/final_lod_movies_dataset.csv",index=False)

In [90]:
final_movies_df_tmp = final_movies_df.copy()

In [91]:
#clean the final dataset
final_movies_df_tmp.replace('NOT_FOUND',np.nan, inplace=True)

In [92]:
x =final_movies_df_tmp.loc[(~(final_movies_df_tmp.genre.isna())
                    & ~(final_movies_df_tmp.actor_name.isna())
                    & ~(final_movies_df_tmp.country.isna())
                    & ~(final_movies_df_tmp.director_name.isna())
                    & ~(final_movies_df_tmp.writer_name.isna()))                        
                ]
len(x)

5597

In [ ]:
final_movies_df_tmp